In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv('Problem_Data/Problem_Data.csv')

In [3]:
df.head()

,primary_id,check_1,check_2,check_3,check_4,check_5,check_6,check_7,check_8,check_9,check_10,some_date
0,REF_0,0,0,0,0,0,0,0,0,0,0,2023-10-26
1,REF_2,0,0,0,0,0,0,0,0,0,0,2023-10-26
2,REF_13,0,0,0,0,0,0,0,1,0,0,2023-10-22
3,REF_14,0,0,0,0,0,0,0,1,0,1,2023-10-24
4,REF_17,0,0,0,0,0,0,1,0,0,1,2023-10-21


In [4]:
# Convert 'some_date' column to datetime type
df['some_date'] = pd.to_datetime(df['some_date'])

In [5]:
check_order = ['total', 'check_4', 'check_3', 'check_10', 'check_1', 'check_6', 
               'check_9', 'check_2', 'check_5', 'check_8', 'check_7']

In [6]:
cols = ['total', 'In Last 0-5 days', 'In Last 6-15 days', 'In Last 16-30 days', 'Before 30 days', 'In Future']

# Initialising empty result df with index and column names
results_df = pd.DataFrame(index=check_order, columns=cols)

- ### Considering that we need consider a user as qualified a check if and only if he passes the current check as well as the checks before it in the ***check_order***

- ### The other version where we don't consider previous checks has also been done at the bottom 

In [7]:
for check in check_order[1:]:
    # Filter rows based on the check order condition
    filter_condition = df[check]
    for prev_check in check_order[1:check_order.index(check)]:
        filter_condition = filter_condition & df[prev_check]

    # Calculate the total count
    total_count = filter_condition.sum()

    # Calculate date-specific counts
    date_counts = []
    date_ranges = [(0, 5), (6, 15), (16, 30)]
    today = datetime.now()
    for date_range in date_ranges:
        start_date = str((today - timedelta(days=date_range[1])).date())
        end_date = str((today - timedelta(days=date_range[0])).date())
        date_filtered_df = df[(df['some_date'] >= start_date) & (df['some_date'] <= end_date) & (filter_condition)]
        date_counts.append(date_filtered_df.shape[0])

    # Calculate counts for Before 30 days, and In Future
    before_30_days = df[(df['some_date'] < str((today - timedelta(days=30)).date())) & filter_condition].shape[0]
    in_future = df[(df['some_date'] > str(today.date())) & filter_condition].shape[0]

    results_df.loc[check] = [total_count] + date_counts + [before_30_days, in_future]

In [8]:
# To calculate the 'total' row
def calculate_total(df):
    total = df.shape[0]
    date_counts = []
    date_ranges = [(0, 5), (6, 15), (16, 30)]
    today = datetime.now()
    for date_range in date_ranges:
        start_date = str((today - timedelta(days=date_range[1])).date())
        end_date = str((today - timedelta(days=date_range[0])).date())
        date_filtered_df = df[(df['some_date'] >= start_date) & (df['some_date'] <= end_date)]
        date_counts.append(date_filtered_df.shape[0])

    # Caluculate counts for Before 30 days, and In Future
    before_30_days = df[(df['some_date'] < str((today - timedelta(days=30)).date()))].shape[0]
    in_future = df[(df['some_date'] > str(today.date()))].shape[0]
    
    return [total] + date_counts + [before_30_days, in_future]

In [9]:
results_df.loc['total'] = calculate_total(df)

In [10]:
results_df

,total,In Last 0-5 days,In Last 6-15 days,In Last 16-30 days,Before 30 days,In Future
total,584166,26656,26736,24330,289536,216908
check_4,4253,208,213,170,1604,2058
check_3,34,0,1,0,6,27
check_10,4,0,0,0,2,2
check_1,0,0,0,0,0,0
check_6,0,0,0,0,0,0
check_9,0,0,0,0,0,0
check_2,0,0,0,0,0,0
check_5,0,0,0,0,0,0
check_8,0,0,0,0,0,0


In [11]:
results_df.to_excel('result_funnel_1.xlsx')

### Not considering previous checks

In [12]:
results_df2 = pd.DataFrame(index=check_order, columns=cols)

In [13]:
for check in check_order[1:]:
    # Now filter condition is only qualifying in currect check
    filter_condition = df[check]

    # Calculate the total count
    total_count = filter_condition.sum()

    # Calculate date-specific counts
    date_counts = []
    date_ranges = [(0, 5), (6, 15), (16, 30)]
    today = datetime.now()
    for date_range in date_ranges:
        start_date = str((today - timedelta(days=date_range[1])).date())
        end_date = str((today - timedelta(days=date_range[0])).date())
        date_filtered_df = df[(df['some_date'] >= start_date) & (df['some_date'] <= end_date) & (filter_condition)]
        date_counts.append(date_filtered_df.shape[0])

    # Calculate counts for Before 30 days, and In Future
    before_30_days = df[(df['some_date'] < str((today - timedelta(days=30)).date())) & filter_condition].shape[0]
    in_future = df[(df['some_date'] > str(today.date())) & filter_condition].shape[0]

    results_df2.loc[check] = [total_count] + date_counts + [before_30_days, in_future]

In [14]:
results_df2.loc['total'] = calculate_total(df)

In [15]:
results_df2

,total,In Last 0-5 days,In Last 6-15 days,In Last 16-30 days,Before 30 days,In Future
total,584166,26656,26736,24330,289536,216908
check_4,4253,208,213,170,1604,2058
check_3,16608,876,770,737,6771,7454
check_10,81255,3508,3163,2957,44371,27256
check_1,46345,851,901,928,36776,6889
check_6,42796,2794,3037,2829,9432,24704
check_9,15646,887,1040,934,1887,10898
check_2,1056,64,48,50,406,488
check_5,46208,1643,1609,1387,29645,11924
check_8,179665,5485,6182,7007,109595,51396


In [17]:
results_df2.to_excel('result_funnel_2.xlsx')